In [88]:
import numpy as np
import pandas as pd
from matplotlib import style
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV


plt.rcParams['figure.figsize'] = (12, 6)
style.use('ggplot')

#X = pd.get_dummies(data.X)
data = pd.read_csv('data/Ffires.csv')
#data[data.area >= 5.0].loc[:, 'area'] = 1
data.describe()

data.at[data.index[data.area < 5.0], 'area'] = 0
data.at[data.index[data.area >= 5.0], 'area'] = 1

freq = Counter(data.area)
freq

#month 와 day도 중요한 변수일 수 있지만 그 지역의 계절적 특성을 보여주는
# 습도, 강수량, 온도 등으로 충분히 어느 시점인지 예측할 수 있을거 같아 제외시켰습니다.
data = data.drop(['month','day'], axis=1);
data

#data scaled
# data 간의 범위의 차이가 있어 min max scaling을 통해 0-1 사이로 수정한다.
# scaler = MinMaxScaler()
# data[data.columns] = scaler.fit_transform(data[data.columns])
data['area'] = data['area'].astype('int')
data['area'] = data['area'].astype('category')

data

y = data['area']
X = data.drop(['area'], axis=1)

dummies_X = pd.get_dummies(X['X']).rename(columns=lambda x: 'X_' + str(x))
dummies_Y = pd.get_dummies(X['Y']).rename(columns=lambda x: 'Y_' + str(x))

X_scaled = pd.DataFrame(MinMaxScaler().fit_transform(X.loc[:, 'FFMC':]), columns=X.loc[:, 'FFMC':].columns)

dummies = pd.concat([dummies_X, dummies_Y], axis=1)

X = pd.concat([X_scaled, dummies], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

def svc_param_selection (X,y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search

best_model = svc_param_selection(X_train, y_train, 10);

clf = best_model.best_estimator_.fit(X_train, y_train)

clf.predict(X_test)

confusion_matrix(clf.predict(X_test), y_test)